# Downloading the pyspark dataset

In [9]:
tweets_df = spark.read.csv('/user1/JoeBidenTweets.csv', header=True, inferSchema=True)
tweets_df.head()

Row(id='361388562', timestamp='2007-10-24 22:45', url='https://twitter.com/JoeBiden/status/361388562', tweet='Tune in 11:30 ET tomorrow for a live webcast of Families USA Presidential Forum on health care: http://presidentialforums.health08.org/', replies='19', retweets='5', quotes='17', likes='11')

# Getting the tweets with maximum tweets

In [10]:
# Create a temporary view of the DataFrame
tweets_df.createOrReplaceTempView("tweets")

# Write the SQL query to find the tweet with maximum likes
result = spark.sql("SELECT * FROM tweets WHERE likes = (SELECT MAX(likes) FROM tweets)")

# View the query result
result.show()


+-------------------+----------------+--------------------+--------------------+-------+--------+------+-----+
|                 id|       timestamp|                 url|               tweet|replies|retweets|quotes|likes|
+-------------------+----------------+--------------------+--------------------+-------+--------+------+-----+
|1149155853095919616|2019-07-11 03:18|https://twitter.c...|Today, we celebra...|     70|     174|    10|  999|
+-------------------+----------------+--------------------+--------------------+-------+--------+------+-----+



# Getting top five liked tweets

In [11]:
# Create a temporary view of the DataFrame
tweets_df.createOrReplaceTempView("tweets")

# Write the SQL query to get the top five tweets with maximum likes
result = spark.sql("SELECT * FROM tweets ORDER BY likes DESC LIMIT 5")

# View the query result
result.show()


+-------------------+----------------+--------------------+--------------------+-------+--------+------+-----+
|                 id|       timestamp|                 url|               tweet|replies|retweets|quotes|likes|
+-------------------+----------------+--------------------+--------------------+-------+--------+------+-----+
|1149155853095919616|2019-07-11 03:18|https://twitter.c...|Today, we celebra...|     70|     174|    10|  999|
|1144700495807881223|2019-06-28 20:14|https://twitter.c...|I’ve fought my he...|   1435|    1772|   199| 9984|
| 864879534096629760|2017-05-17 16:25|https://twitter.c...|LGBT rights are h...|    247|    2995|   157| 9980|
|1243255073817460741|2020-03-26 19:14|https://twitter.c...|Live from our hom...|    537|    2290|    66| 9975|
| 958553443429478400|2018-01-31 04:12|https://twitter.c...|Proud to call you...|   1480|   18794|   513|99747|
+-------------------+----------------+--------------------+--------------------+-------+--------+------+-----+



# Some queries regarding the dataset

In [15]:
# Create a temporary view of the DataFrame
tweets_df.createOrReplaceTempView("tweets")

# Perform queries using PySpark SQL
# Example 1: Retrieve tweets with a specific number of likes
query1 = """
SELECT * 
FROM tweets 
WHERE likes = 100
"""

result1 = spark.sql(query1)
result1.show()

# Example 2: Calculate the total number of likes
query2 = """
SELECT SUM(likes) AS total_likes
FROM tweets
"""

result2 = spark.sql(query2)
result2.show()

# Example 3: Sort tweets based on the number of likes
query3 = """
SELECT *
FROM tweets
ORDER BY likes DESC
"""

result3 = spark.sql(query3)
result3.show()



# Example 5: Perform text analysis - count the occurrences of specific words
query4 = """
SELECT word, COUNT(*) AS word_count
FROM (
    SELECT EXPLODE(SPLIT(tweet, ' ')) AS word
    FROM tweets
) temp
GROUP BY word
"""


query4 = spark.sql(query4)
result4.show()


+------------------+----------------+--------------------+--------------------+-------+--------+------+-----+
|                id|       timestamp|                 url|               tweet|replies|retweets|quotes|likes|
+------------------+----------------+--------------------+--------------------+-------+--------+------+-----+
|260911167852142592|2012-10-24 01:10|https://twitter.c...|“I’ve got news fo...|     96|     527|     1|  100|
|263097044196028416|2012-10-30 01:56|https://twitter.c...|.@BarackObama and...|     54|     215|     1|  100|
|456901642026508288|2014-04-17 21:06|https://twitter.c...|RT @TheDemocrats:...|     34|      87|     0|  100|
|509016433851121665|2014-09-08 16:32|https://twitter.c...|Read the VP's op-...|     41|      83|     2|  100|
+------------------+----------------+--------------------+--------------------+-------+--------+------+-----+

+------------+
| total_likes|
+------------+
|1.51694453E8|
+------------+

+-------------------+----------------+-----

# Creating scemas with this dataset 

In [5]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# Define the schema for the DataFrame
schema = StructType([
    StructField("id", StringType(), nullable=True),
    StructField("tweet", StringType(), nullable=True),
    StructField("url", StringType(), nullable=True),
    
    StructField("replies", IntegerType(), nullable=True),
    StructField("quotes", IntegerType(), nullable=True),
    StructField("retweets", IntegerType(), nullable=True)
])

# Apply the schema to the DataFrame
tweets_df = spark.read.csv('/user1/JoeBidenTweets.csv', header=True, schema=schema)


In [6]:
tweets_df.show()


+------------------+----------------+--------------------+-------+------+--------+
|                id|           tweet|                 url|replies|quotes|retweets|
+------------------+----------------+--------------------+-------+------+--------+
|         361388562|2007-10-24 22:45|https://twitter.c...|   null|    19|       5|
|         543984392|2007-12-29 15:35|https://twitter.c...|   null|    13|      16|
|189287227321356289|2012-04-09 09:42|https://twitter.c...|   null|    21|      82|
|189287350034104320|2012-04-09 09:43|https://twitter.c...|   null|   144|      76|
|189339650610036736|2012-04-09 13:11|https://twitter.c...|   null|    10|      54|
|189343279140184065|2012-04-09 13:25|https://twitter.c...|   null|    16|      52|
|189383473717460992|2012-04-09 16:05|https://twitter.c...|   null|   114|     471|
|189456416917032960|2012-04-09 20:55|https://twitter.c...|   null|    13|      29|
|189712253187145728|2012-04-10 13:51|https://twitter.c...|   null|    21|     217|
|189

2023-05-25 11:44:44,602 WARN csv.CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 8, schema size: 6
CSV file: hdfs://localhost:9000/user1/JoeBidenTweets.csv


# Extracting hashtages

In [17]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType
import re

# Create a SparkSession
spark = SparkSession.builder.getOrCreate()

# Load the dataset into a DataFrame
df = spark.read.csv("/user1/JoeBidenTweets.csv", header=True, inferSchema=True)

# UDF for extracting hashtags
def extract_hashtags(tweet):
    hashtags = re.findall(r'#(\w+)', tweet)
    return hashtags

# Register UDF
extract_hashtags_udf = udf(extract_hashtags, ArrayType(StringType()))

# Apply UDF to extract hashtags
df_hashtags = df.select(explode(extract_hashtags_udf("tweet")).alias("hashtag"))

# Show the extracted hashtags
df_hashtags.show()


+-----------+
|    hashtag|
+-----------+
|BuffettRule|
|BuffettRule|
|BuffettRule|
|     Exeter|
|BuffettRule|
|BuffettRule|
|BuffettRule|
|BuffettRule|
|BuffettRule|
|BuffettRule|
|      Gen44|
|BuffettRule|
|BuffettRule|
|      Gen44|
|Latinos2012|
|      Gen44|
|       VAWA|
|       VAWA|
|       VAWA|
|  Obama2012|
+-----------+
only showing top 20 rows

